# TensorRT

In [1]:
import pathlib
import os
import time
import numpy as np
import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import load_model

2024-10-24 19:49:00.342419: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 19:49:00.352145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 19:49:00.362564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 19:49:00.365663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 19:49:00.374597: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


I0000 00:00:1729813741.212840   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.213016   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.243770   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.243973   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14603864997363321543
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22914203648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4160016612379630869
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:41:00.0, compute capability: 8.9"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23354408960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4831840866701438019
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:82:00.0, compute capability: 8.9"
xla_global_id: 2144165316
]


I0000 00:00:1729813741.368871   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369036   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369146   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369247   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

(8, 6, 1)


### Benchmarking

In [8]:
# load model
model = tf.keras.models.load_model("../best_model/mobilenetv2_best_f1score_fold_1.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../best_model/mobilenetv2_best_f1score_fold_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
data_root = pathlib.Path("../data/Original_Images")    # points to the folder containing the images that will be used for training

# hyperparameters
batch_size = 32         # size of the batch that will be fed to model
img_height = 224        # input image height
img_width = 224         # input image width
test_size = 0.2

# Load dataset without splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,                                  # loads images from the data_root directory
    image_size=(img_height, img_width),         # resizes all images to (224, 224) pixels
    batch_size=batch_size,                      # set the batch size
    shuffle=True                                # shufle data when loaded
)

# Preprocess dataset (we only need the images for inference)
image_batches = []
for image_batch, _ in dataset:
    image_batches.append(image_batch)

image_batches = np.concatenate(image_batches)  # Flatten batches to get all images
print(f"Total images: {image_batches.shape[0]}")

# Split the data into test subset for benchmarking
_, X_test = train_test_split(image_batches, test_size=test_size, random_state=42)

Found 228 files belonging to 2 classes.
Total images: 228


In [6]:
# run inference
trials = 1000
inference_times = []

print(f"Running {trials} inference trials on {len(X_test)} test images...")
for i in range(trials):
    start_time = time.perf_counter()

    # Run inference for a batch of images
    model.predict(X_test, batch_size=batch_size, verbose=0)

    end_time = time.perf_counter()
    inference_time = end_time - start_time
    inference_times.append(inference_time)

    if i % 50 == 0:
        avg_inference = np.mean(inference_times)  # Average inference time per trial
        print(f"Step {i}: average inference time = {avg_inference:.6f} seconds")
        
    tf.keras.backend.clear_session()
        
# Compute throughput (images per second)
total_time = np.sum(inference_times)
throughput = (trials * len(X_test)) / total_time
print(f"Throughput: {throughput:.2f} images/second")

Running 1000 inference trials on 46 test images...
Step 0: average inference time = 0.197487 seconds
Step 50: average inference time = 0.530172 seconds
Step 100: average inference time = 0.532068 seconds
Step 150: average inference time = 0.534038 seconds
Step 200: average inference time = 0.538387 seconds
Step 250: average inference time = 0.542415 seconds
Step 300: average inference time = 0.544953 seconds
Step 350: average inference time = 0.546347 seconds
Step 400: average inference time = 0.548982 seconds
Step 450: average inference time = 0.551061 seconds
Step 500: average inference time = 0.551467 seconds
Step 550: average inference time = 0.552484 seconds
Step 600: average inference time = 0.553712 seconds
Step 650: average inference time = 0.554324 seconds
Step 700: average inference time = 0.555828 seconds
Step 750: average inference time = 0.555979 seconds
Step 800: average inference time = 0.555808 seconds
Step 850: average inference time = 0.555597 seconds
Step 900: averag

### TensorRT Optimization

In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.TrtConversionParams(
    precision_mode=trt.TrtPrecisionMode.FP32)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir='../best_model/model1/mobilenetv2_best_f1score_fold',
    conversion_params=conversion_params)

# Convert the model
converter.convert()
converter.summary()

# Save the optimized model
converter.save('../tensorRT_model/test')

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


2024-10-24 19:42:51.713773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RuntimeError: Tensorflow has not been built with TensorRT support.

### Run Inference on Optimized Model

In [ ]:
# # Load the optimized TensorRT model
# saved_model_loaded = tf.saved_model.load('path/to/save/tensorrt_model')
# infer = saved_model_loaded.signatures['serving_default']

# # Example input data (adjust as per your model's input requirements)
# input_tensor = tf.convert_to_tensor(your_input_data)

# # Run inference
# output = infer(input_tensor)

# # Process output as needed